In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow.keras.preprocessing.image import DirectoryIterator
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

def delete_corrupted_images(dataset_folder):
    # Walk through the directory and subdirectories
    for root, dirs, files in os.walk(dataset_folder):
        for file in files:
            # Construct the full image path
            file_path = os.path.join(root, file)
            try:
                # Try loading the image
                img = load_img(file_path)
                img_to_array(img)  # This will raise an error if the image is corrupted
            except (OSError, ValueError) as e:
                print(f"Error loading image {file_path}: {e}. Deleting the image.")
                os.remove(file_path)  # Delete the corrupted image

# Specify the folder containing dataset
dataset_folder = r"C:/Users/ASUS/Desktop/Dataset"  
delete_corrupted_images(dataset_folder)

class CustomDirectoryIterator(DirectoryIterator):
    def _load_image(self, path):
        try:
            # Attempt to load the image
            img = Image.open(path)
            img = img.convert('RGB')  
            img = img.resize((224, 224))  # Resize to the target size (224x224)
            img = np.array(img) / 255.0  # Normalize the image
            if img.shape != (224, 224, 3):  # Ensured the correct shape
                return None
            return img
        except (IOError, SyntaxError, OSError) as e:
            # Print error message and skip corrupted image
            print(f"Skipping corrupted image: {path}, Error: {e}")
            return None  # Return None for corrupted images

    def next(self):
        # Override next() method to skip corrupted images
        batch_x, batch_y = [], []
        while len(batch_x) < self.batch_size:
            x, y = super().next()  # Get the next batch
            if x is not None:  # Skip corrupted images
                batch_x.append(x)
                batch_y.append(y)
        return np.array(batch_x), np.array(batch_y)

dataset_dir = r"C:/Users/ASUS/Desktop/Dataset"  

# Create the ImageDataGenerator for loading and rescaling images
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Custom iterator to load images
train_data = CustomDirectoryIterator(
    dataset_dir,
    datagen,
    target_size=(224, 224),  
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Found 1080 images belonging to 5 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

# Load a pre-trained model (MobileNetV2)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
base_model.trainable = False

# Add custom layers for yoga pose classification
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dense(train_data.num_classes, activation='softmax')  # Output layer with number of classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_data, epochs=10, steps_per_epoch=train_data.samples // train_data.batch_size)

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 32s 840ms/step - accuracy: 0.5334 - loss: 1.3344
Epoch 2/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 872us/step - accuracy: 0.8750 - loss: 0.3996 
Epoch 3/10


c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


33/33 ━━━━━━━━━━━━━━━━━━━━ 28s 857ms/step - accuracy: 0.9161 - loss: 0.2753
Epoch 4/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 292us/step - accuracy: 0.9375 - loss: 0.2021 
Epoch 5/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 27s 813ms/step - accuracy: 0.9602 - loss: 0.1571
Epoch 6/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 0s/step - accuracy: 1.0000 - loss: 0.0422    
Epoch 7/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 27s 801ms/step - accuracy: 0.9892 - loss: 0.0687
Epoch 8/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 0s/step - accuracy: 1.0000 - loss: 0.0394    
Epoch 9/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 26s 775ms/step - accuracy: 0.9927 - loss: 0.0468
Epoch 10/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 0s/step - accuracy: 1.0000 - loss: 0.0462    


In [ ]:
import cv2
import mediapipe as mp
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Initialize MediaPipe Pose detector
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Class labels corresponding to your trained model's output
class_labels = ["Downdog", "Goddess", "Plank", "Tree", "Warrior2"]

# Define the ideal pose details
ideal_pose_landmarks = {
    "Downdog": {
        "hips": {"y_ratio": 1.2}, 
        "spine": {"straight": True},
        "arms": {"straight": True},
        "knees": {"angle": [170, 180]},
        "feet": {"alignment": "grounded"},
    },
    "Goddess": {
        "knees": {"angle": [80, 100]},
        "spine": {"upright": True},
        "arms": {"horizontal": True},
        "feet": {"angle_outward": True},
    },
    "Plank": {
        "spine": {"straight": True},
        "hips": {"alignment": "neutral"},
        "arms": {"vertical": True},
    },
    "Tree": {
        "hips": {"level": True},
        "standing_leg": {"straight": True},
        "lifted_leg": {"placement": "above_or_below_knee"},
        "arms": {"position": ["above_head", "folded"]},
    },
    "Warrior2": {
        "front_knee": {"angle": [80, 100]},
        "back_leg": {"straight": True},
        "arms": {"horizontal": True},
        "hips_shoulders": {"alignment": "forward"},
    },
}

# Calculate angles between three points
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(cosine_angle))

# Provide feedback based on pose and landmark analysise
def evaluate_pose(pose_label, landmarks):
    feedback = []
    if pose_label in ideal_pose_landmarks:
        ideal = ideal_pose_landmarks[pose_label]

        # **Hips Evaluation**
        if "hips" in ideal and "y_ratio" in ideal["hips"]:
            hip_y = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
            shoulder_y = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            if hip_y < shoulder_y * ideal["hips"]["y_ratio"]:
                feedback.append("Raise your hips higher.")

        # **Spine Evaluation**
        if "spine" in ideal and "straight" in ideal["spine"]:
            spine_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x, landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y],
            )
            if spine_angle < 160:
                feedback.append("Keep your spine straight.")

        # **Arm Evaluation**
        if "arms" in ideal:
            left_arm_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y],
            )
            right_arm_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y],
            )
            if "straight" in ideal["arms"]:
                if not (160 <= left_arm_angle <= 180):
                    feedback.append("Straighten your left arm.")
                if not (160 <= right_arm_angle <= 180):
                    feedback.append("Straighten your right arm.")
            if "horizontal" in ideal["arms"]:
                if abs(left_arm_angle - 90) > 15 or abs(right_arm_angle - 90) > 15:
                    feedback.append("Keep your arms horizontal.")

        # **Knee Evaluation**
        if "knees" in ideal and "angle" in ideal["knees"]:
            left_knee_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y],
            )
            right_knee_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y],
            )
            ideal_knee_range = ideal["knees"]["angle"]
            if not (ideal_knee_range[0] <= left_knee_angle <= ideal_knee_range[1]):
                feedback.append("Adjust your left knee angle.")
            if not (ideal_knee_range[0] <= right_knee_angle <= ideal_knee_range[1]):
                feedback.append("Adjust your right knee angle.")

        # **Feet Evaluation**
        if "feet" in ideal:
            if "grounded" in ideal["feet"]:
                left_foot_y = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
                right_foot_y = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y
                if abs(left_foot_y - right_foot_y) > 0.05:  # Threshold for uneven feet
                    feedback.append("Keep your feet grounded evenly.")
            if "alignment" in ideal["feet"]:
                left_foot_alignment = calculate_angle(
                    [landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y],
                )
                right_foot_alignment = calculate_angle(
                    [landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y],
                )
                if not (80 <= left_foot_alignment <= 100):
                    feedback.append("Adjust your left foot alignment.")
                if not (80 <= right_foot_alignment <= 100):
                    feedback.append("Adjust your right foot alignment.")


    return feedback


# Process video input for pose classification and feedback
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform pose detection
    results = pose.process(rgb_frame)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        landmarks = results.pose_landmarks.landmark

        # Predict the yoga pose
        resized_frame = cv2.resize(frame, (224, 224))
        processed_frame = np.expand_dims(resized_frame / 255.0, axis=0)
        predictions = model.predict(processed_frame)
        pose_label = class_labels[np.argmax(predictions)]

        # Provide feedback
        feedback = evaluate_pose(pose_label, landmarks)

        # Display results
        cv2.putText(frame, f"Pose: {pose_label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        for i, line in enumerate(feedback):
            cv2.putText(frame, line, (10, 70 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    cv2.imshow("Yoga Pose Feedback", frame)

    #Press 'a' to end the live detection and feedback
    if cv2.waitKey(10) & 0xFF == ord('a'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━